In [2]:
import pandas as pd
from sodapy import Socrata
import case_3_api as c3api


In [4]:

results_df = c3api.get_RDW_brandstof(limit=3000000,offset=0, where="brandstof_omschrijving='Elektriciteit'")
print(results_df.shape)
results_df.head()

(10, 7)


,kenteken,brandstof_volgnummer,brandstof_omschrijving,emissiecode_omschrijving,uitlaatemissieniveau,opgegeven_maximum_snelheid,nettomaximumvermogen
0,0001TJ,1,Benzine,0,EURO 0,NaN,NaN
1,0001TV,1,Benzine,0,EURO 0,NaN,NaN
2,0001VH,1,Benzine,0,NaN,NaN,NaN
3,0001VM,1,Benzine,0,EURO 0,NaN,NaN
4,0002GV,1,Diesel,NaN,NaN,25.00,NaN


In [18]:
len(results_df)

1224231

In [10]:
df = c3api.get_RDW_brandstof(500000)

In [11]:
df.value_counts('brandstof_omschrijving')

brandstof_omschrijving
Benzine          413537
Diesel            71486
LPG                7581
Elektriciteit      6786
Alcohol             259
CNG                 208
LNG                 134
Waterstof             9
dtype: int64

In [2]:

print(c3api.get_RDW_brandstof(100).head())
print(c3api.get_RDW_kenteken_df(100).head())
print(c3api.get_OCM_df(100).head())

  kenteken brandstof_volgnummer brandstof_omschrijving  \
0   0001TJ                    1                Benzine   
1   0001TV                    1                Benzine   
2   0001VH                    1                Benzine   
3   0001VM                    1                Benzine   
4   0002GV                    1                 Diesel   

  emissiecode_omschrijving uitlaatemissieniveau opgegeven_maximum_snelheid  \
0                        0               EURO 0                        NaN   
1                        0               EURO 0                        NaN   
2                        0                  NaN                        NaN   
3                        0               EURO 0                        NaN   
4                      NaN                  NaN                      25.00   

  nettomaximumvermogen geluidsniveau_rijdend geluidsniveau_stationair  \
0                  NaN                   NaN                      NaN   
1                  NaN              

In [26]:
client = Socrata("opendata.rdw.nl", None)
results = client.get("m9d7-ebf2", limit=2000,offset=1)
results_df = pd.DataFrame.from_records(results)
results_df.shape

(2000, 67)

In [17]:
len(client.get_metadata("m9d7-ebf2")['columns'])

92